In [ ]:
import cv2
import dlib
import numpy as np
from keras.models import load_model
from scipy.spatial import distance as dist
from imutils import face_utils

predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')

# detect the face rectangle 
def detect(img, cascade = face_cascade , minimumFeatureSize=(20, 20)):
    if cascade.empty():
        raise (Exception("There was a problem loading your Haar Cascade xml file."))
    rects = cascade.detectMultiScale(img, scaleFactor=1.3, minNeighbors=1, minSize=minimumFeatureSize)
    
    # if it doesn't return rectangle return array
    # with zero lenght
    if len(rects) == 0:
        return []

    #  convert last coord from (width,height) to (maxX, maxY)
    rects[:, 2:] += rects[:, :2]

    return rects

def cropEyes(frame):
	 
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
	
	# detect the face at grayscale image
	te = detect(gray, minimumFeatureSize=(80, 80))

	# if the face detector doesn't detect face
	# return None, else if detects more than one faces
	# keep the bigger and if it is only one keep one dim
	if len(te) == 0:
		return None
	elif len(te) > 1:
		face = te[0]
	elif len(te) == 1:
		[face] = te

	# keep the face region from the whole frame
	face_rect = dlib.rectangle(left = int(face[0]), top = int(face[1]),
								right = int(face[2]), bottom = int(face[3]))
	
	# determine the facial landmarks for the face region
	shape = predictor(gray, face_rect)
	shape = face_utils.shape_to_np(shape)

	#  grab the indexes of the facial landmarks for the left and
	#  right eye, respectively
	(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
	(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

	# extract the left and right eye coordinates
	leftEye = shape[lStart:lEnd]
	rightEye = shape[rStart:rEnd]

	# keep the upper and the lower limit of the eye 
	# and compute the height 
	l_uppery = min(leftEye[1:3,1])
	l_lowy = max(leftEye[4:,1])
	l_dify = abs(l_uppery - l_lowy)

	# compute the width of the eye
	lw = (leftEye[3][0] - leftEye[0][0])

	# we want the image for the cnn to be (26,34)
	# so we add the half of the difference at x and y
	# axis from the width at height respectively left-right
	# and up-down 
	minxl = (leftEye[0][0] - ((34-lw)/2))
	maxxl = (leftEye[3][0] + ((34-lw)/2)) 
	minyl = (l_uppery - ((26-l_dify)/2))
	maxyl = (l_lowy + ((26-l_dify)/2))
	
	# crop the eye rectangle from the frame
	left_eye_rect = np.rint([minxl, minyl, maxxl, maxyl])
	left_eye_rect = left_eye_rect.astype(int)
	left_eye_image = gray[(left_eye_rect[1]):left_eye_rect[3], (left_eye_rect[0]):left_eye_rect[2]]
	
	# same as left eye at right eye
	r_uppery = min(rightEye[1:3,1])
	r_lowy = max(rightEye[4:,1])
	r_dify = abs(r_uppery - r_lowy)
	rw = (rightEye[3][0] - rightEye[0][0])
	minxr = (rightEye[0][0]-((34-rw)/2))
	maxxr = (rightEye[3][0] + ((34-rw)/2))
	minyr = (r_uppery - ((26-r_dify)/2))
	maxyr = (r_lowy + ((26-r_dify)/2))
	right_eye_rect = np.rint([minxr, minyr, maxxr, maxyr])
	right_eye_rect = right_eye_rect.astype(int)
	right_eye_image = gray[right_eye_rect[1]:right_eye_rect[3], right_eye_rect[0]:right_eye_rect[2]]

	# if it doesn't detect left or right eye return None
	if 0 in left_eye_image.shape or 0 in right_eye_image.shape:
		return None
	# resize for the conv net
	left_eye_image = cv2.resize(left_eye_image, (34, 26))
	right_eye_image = cv2.resize(right_eye_image, (34, 26))
	right_eye_image = cv2.flip(right_eye_image, 1)
	# return left and right eye
	return left_eye_image, right_eye_image 

# make the image to have the same format as at training 
def cnnPreprocess(img):
	img = img.astype('float32')
	img /= 255
	img = np.expand_dims(img, axis=2)
	img = np.expand_dims(img, axis=0)
	return img

def main():
	# open the camera,load the cnn model 
	camera = cv2.VideoCapture('video.mp4')
	model = load_model('blinkModel.hdf5')
	
	# blinks is the number of total blinks ,close_counter
	# the counter for consecutive close predictions
	# and mem_counter the counter of the previous loop 
	close_counter = blinks = mem_counter= 0
	state = ''
	while True:
		
		ret, frame = camera.read()
		
		# detect eyes
		eyes = cropEyes(frame)
		if eyes is None:
			continue
		else:
			left_eye,right_eye = eyes
		
		# average the predictions of the two eyes 
		prediction = (model.predict(cnnPreprocess(left_eye)) + model.predict(cnnPreprocess(right_eye)))/2.0
			
		# blinks
		# if the eyes are open reset the counter for close eyes
		if prediction > 0.5 :
			state = 'open'
			close_counter = 0
		else:
			state = 'close'
			close_counter += 1
		
		# if the eyes are open and previousle were closed
		# for sufficient number of frames then increcement 
		# the total blinks
		if state == 'open' and mem_counter > 1:
			blinks += 1
		# keep the counter for the next loop 
		mem_counter = close_counter 

		# draw the total number of blinks on the frame along with
		# the state for the frame
		cv2.putText(frame, "Blinks: {}".format(blinks), (10, 30),
			cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
		cv2.putText(frame, "State: {}".format(state), (300, 30),
			cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
		
		# show the frame
# 		cv2.imshow('blinks counter', frame)
		key = cv2.waitKey(1) & 0xFF

		# if the `q` key was pressed, break from the loop
		if key == ord('q'):
			break
	# do a little clean up
	cv2.destroyAllWindows()
	del(camera)


if __name__ == '__main__':
	main()

Using TensorFlow backend.


RuntimeError: ignored

In [ ]:

#first install mtcnn using  ->      !pip install mtcnn
#Then download and upload “shape_predictor_68_face_landmarks.dat”  file
#upload the attached video also

import cv2

import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
from mtcnn.mtcnn import MTCNN
import numpy as np
from scipy.spatial import distance
from imutils import face_utils
import imutils
import dlib


def eye_aspect_ratio(eye):
                A = distance.euclidean(eye[1], eye[5])
                B = distance.euclidean(eye[2], eye[4])
                C = distance.euclidean(eye[0], eye[3])
                ear = (A + B) / (2.0 * C)
                return ear

def detect_face(original_frame):
  
    detector = MTCNN()
    original_image = cv2.imread(original_frame)
    resultant_json_object = detector.detect_faces(original_image)
    print(resultant_json_object)
    duplicate_image=original_image.copy()
    
    index=len(resultant_json_object)
    print(index)                  #It'll display count of total face detected. i.e total number of bounding_boxes found
    
    if(index==0):
      print("No face detected")
      return ;
    

    for i in range(0,index):
      bounding_box = resultant_json_object[i]['box']
#       keypoints = resultant_json_object[i]['keypoints']
      confidence=resultant_json_object[i]['confidence']   #accuracy of detected face
      cv2.rectangle(original_image,(bounding_box[0],bounding_box[1]),(bounding_box[0]+bounding_box[2], bounding_box[1] + bounding_box[3]),(0,255,0),3)
#       cv2.circle(original_image,(keypoints['left_eye']), 2, (0,155,255), 2)
#       cv2.circle(original_image,(keypoints['right_eye']), 2, (0,155,255), 2)
#       cv2.circle(original_image,(keypoints['nose']), 2, (0,155,255), 2)
#       cv2.circle(original_image,(keypoints['mouth_left']), 2, (0,155,255), 2)
#       cv2.circle(original_image,(keypoints['mouth_right']), 2, (0,155,255), 2)

      font = cv2.FONT_HERSHEY_DUPLEX 
      cv2.putText(original_image, str(confidence)[0:7], (bounding_box[0],bounding_box[1]), font,0.75, (0, 0,255),2)
      cv2.imwrite("Face_detected_image.jpg", original_image)

    pupil_detect("Face_detected_image.jpg",duplicate_image)

  
def pupil_detect(Face_detected_image,duplicate_image):
  
  Face_detected_frame=cv2.imread(Face_detected_image)
  thresh = 0.25                                    #if ear is less than threshold value then eyes are considered as closed
#   frame_check = 20
  detect = dlib.get_frontal_face_detector()        #return object for face detection          
  predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
  # print(predict)
  (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
  (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]  


  # frame = imutils.resize(frame, width=650)
  gray_image = cv2.cvtColor(Face_detected_frame, cv2.COLOR_BGR2GRAY)
  subjects = detect(gray_image, 0)
  print(subjects)
  if(len(subjects)==0):
    print("Eyes not detected")
    return;
  f, ax1=plt.subplots(2,1)
  for subject in subjects:
    shape = predict(gray, subject)
    shape = face_utils.shape_to_np(shape)          #converting to NumPy Array
    leftEye = shape[lStart:lEnd]
    rightEye = shape[rStart:rEnd]
    print("leftEye co-ordinates ",leftEye)
    print("rightEye co-ordinates",rightEye)
    
    leftEyeXMid=(leftEye[1][0]+leftEye[5][0]+leftEye[2][0]+leftEye[4][0])//4
    leftEyeYMid=(leftEye[1][1]+leftEye[5][1]+leftEye[2][1]+leftEye[4][1])//4

    rightEyeXMid=(rightEye[1][0]+rightEye[5][0]+rightEye[2][0]+rightEye[4][0])//4
    rightEyeYMid=(rightEye[1][1]+rightEye[5][1]+rightEye[2][1]+rightEye[4][1])//4

  #   print(leftEye[1][0],leftEye[1][1])
  #   print(leftEye[5][0],leftEye[5][1])
  #   print(leftEye[2][0],leftEye[2][1])
  #   print(leftEye[4][0],leftEye[4][1])
  #   print(leftEyeXMid)
  #   print(leftEyeYMid)
  
    leftEAR = eye_aspect_ratio(leftEye)
    rightEAR = eye_aspect_ratio(rightEye)
    ear = (leftEAR + rightEAR) / 2.0
    
  #   print(ear)
  #   print("left ear", leftEAR)
  #   print("right ear", rightEAR)
  #   print(leftEye)
    leftEyeHull = cv2.convexHull(leftEye)
    rightEyeHull = cv2.convexHull(rightEye)
    
    cv2.drawContours(Face_detected_frame, [leftEyeHull], -1, (0, 255, 0), 1)
    cv2.drawContours(Face_detected_frame, [rightEyeHull], -1, (0, 255, 0), 1)


    if ear < thresh:                          #To check whether eyes are closed or not
      
      cv2.putText(Face_detected_frame, "****************Eyes Closed!****************", (10, 30),
          cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    else:
      
      cv2.circle(Face_detected_frame,(leftEyeXMid,leftEyeYMid), 1, (0,155,255), 1)
      cv2.circle(Face_detected_frame,(rightEyeXMid,rightEyeYMid), 1, (0,155,255), 1)
  # cv2.imshow("Frame", frame)
  
  cv2.imwrite("Pupil_detected_image.jpg",Face_detected_frame)
  img_pupil=mpimg.imread('Pupil_detected_image.jpg')
  ax1[0].imshow(duplicate_image)
  ax1[1].imshow(img_pupil)
  

vidcap = cv2.VideoCapture('Video.mp4')
count = 0
success = True
fps = int(vidcap.get(cv2.CAP_PROP_FPS))

while success:
    success,image = vidcap.read()
    count += 1
#     print('read a new frame:',success)
    if(count==600):
#       cv2.imwrite("frame%d.jpg" % count, image)
      cv2.imwrite("original_frame.jpg", image)
      detect_face("original_frame.jpg")
        
      break;
    
#     if count%(10*fps) == 0 :
#          cv2.imwrite('frame%d.jpg'%count,image)
#          print('successfully written 10th frame')
#     count+=1
#     cv2.imwrite("frame%d.jpg" % count, image)     # save frame as JPEG file
#     cv2.waitKey(2)
    
  
print("Successfully Processed!")

